# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = 'C:/Users/rachi/Downloads/train-data.txt'
TEST_DATA_FILE = 'C:/Users/rachi/Downloads/test-data.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype = int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter = ' ', dtype = int)

In [5]:
sparse_train_data[:5]

array([[   0, 1756,    0,    1],
       [   1,    0,    0,    1],
       [   1,    1,    0,   17],
       [   1,    4,    0,    2],
       [   1,    5,    0,   10]])

In [6]:
sparse_train_data[-5:]

array([[33715,   490,     1,     1],
       [33715,   588,     1,     1],
       [33715,   632,     1,     1],
       [33715,  1114,     1,     1],
       [33715,  1512,     1,     1]])

In [7]:
print('Number of rows in training file: ', sparse_train_data.shape[0])
print('Number of rows in test file: ', sparse_test_data.shape[0])

Number of rows in training file:  1376622
Number of rows in test file:  589487


In [8]:
print('Number of emails in training file:', np.unique(sparse_train_data[:,0]).size)

Number of emails in training file: 23512


In [9]:
print('Number of emails in test file:', np.unique(sparse_test_data[:,0]).size)

Number of emails in test file: 10077


# Create a Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([    0,     1,     2, ..., 33712, 33713, 33715])

In [13]:
full_train_data = pd.DataFrame(index = index_names, columns = column_names)
full_train_data.fillna(value=0, inplace = True)

C:\Users\rachi\AppData\Local\Temp\ipykernel_15048\2370846799.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_train_data.fillna(value=0, inplace = True)


In [14]:
full_train_data.head()

DOC_ID  CATEGORY  0  1  2  3  4  5  6  7  ...  2490  2491  2492  2493  \
0       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
1       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
5       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
9       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

   2494  2495  2496  2497  2498  2499  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     0     0     0     0  
5     0     0     0     0     0     0  
9     0     0     0     0     0     0  

[5 rows x 2502 columns]

# Create a Full Matrix from a Sparse Matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value=0, inplace = True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace = True)
    return full_matrix

In [16]:
%%time

full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

C:\Users\rachi\AppData\Local\Temp\ipykernel_15048\630567689.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace = True)


CPU times: total: 59.2 s
Wall time: 1min 22s


In [17]:
full_train_data.head()

CATEGORY  0   1  2  3  4   5  6  7  8  ...  2490  2491  2492  2493  \
DOC_ID                                         ...                           
0              0  0   0  0  0  0   0  0  0  0  ...     0     0     0     0   
1              0  1  17  0  0  2  10  1  1  0  ...     0     0     0     0   
2              0  0   0  0  0  0   0  0  0  0  ...     0     0     0     0   
5              0  0   0  0  0  1   0  0  0  0  ...     0     0     0     0   
9              0  4  23  0  0  3  12  2  4  0  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
0          0     0     0     0     0     0  
1          0     0     0     0     0     0  
2          0     0     0     0     0     0  
5          0     0     0     0     0     0  
9          0     0     0     0     0     0  

[5 rows x 2501 columns]

In [18]:
full_train_data.tail()

CATEGORY  0  1  2  3  4  5  6  7  8  ...  2490  2491  2492  2493  \
DOC_ID                                       ...                           
33707          1  0  0  0  1  3  0  0  1  0  ...     0     0     0     0   
33709          1  0  0  0  2  0  0  2  0  0  ...     0     0     0     0   
33712          1  0  0  0  1  0  0  1  0  2  ...     0     0     0     0   
33713          1  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
33715          1  0  0  0  1  0  0  0  0  0  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
33707      0     0     0     0     0     0  
33709      0     0     0     0     0     0  
33712      0     0     0     0     0     0  
33713      1     0     0     0     0     0  
33715      0     0     0     0     0     0  

[5 rows x 2501 columns]

# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [20]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is: ', prob_spam)

Probability of spam is:  0.506890098673018


## Total Number of Words/Tokens

In [21]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
0          0     0     0     0     0     0     0     0     0     0  ...     0   
1          1    17     0     0     2    10     1     1     0     0  ...     0   
2          0     0     0     0     0     0     0     0     0     0  ...     0   
5          0     0     0     0     1     0     0     0     0     0  ...     0   
9          4    23     0     0     3    12     2     4     0     1  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
0          0     0     0     0     0     0     0     0     0  
1          0     0     0     0     0     0     0     0     0  
2          0     0     0     0     0     0     0     0     0  
5          0     0     0     0     0     0     0     0     0  
9          0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [22]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(23512,)

In [23]:
email_lengths[:5]

DOC_ID
0      1
1    168
2      5
5     40
9    255
dtype: int64

In [24]:
total_wc = email_lengths.sum()
total_wc

np.int64(2487401)

# Number of Tokens in Spam and Ham Emails

In [25]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(11918,)

In [26]:
spam_wc = spam_lengths.sum()
spam_wc

np.int64(1014340)

In [27]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(11594,)

In [28]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

np.int64(1473061)

In [29]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [30]:
spam_wc + nonspam_wc - total_wc

np.int64(0)

# Summing the Tokens occurring in Spam

In [31]:
full_train_features.shape

(23512, 2500)

In [32]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
3672       0     0     0     0     0     0     1     0     0     0  ...     0   
3674       0     0     0     0     0     0     0     0     0     0  ...     0   
3675       0     0     0     0     1     0     0     0     0     0  ...     0   
3676       0     0     3     1     1     0     1     0     0     2  ...     0   
3677       0     0     0     0     0     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
3672       0     0     0     0     0     0     0     0     0  
3674       0     0     0     0     0     0     0     0     0  
3675       0     0     0     0     0     0     0     0     0  
3676       0     0     0     0     0     0     0     0     0  
3677       0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [33]:
train_spam_tokens.tail()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
33707      0     0     0     1     3     0     0     1     0     1  ...     0   
33709      0     0     0     2     0     0     2     0     0     0  ...     0   
33712      0     0     0     1     0     0     1     0     2     0  ...     0   
33713      0     0     0     0     0     0     0     0     0     0  ...     0   
33715      0     0     0     1     0     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
33707      0     0     0     0     0     0     0     0     0  
33709      0     0     0     0     0     0     0     0     0  
33712      0     0     0     0     0     0     0     0     0  
33713      0     0     0     1     0     0     0     0     0  
33715      0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [34]:
train_spam_tokens.shape

(11918, 2500)

In [35]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1
summed_spam_tokens.shape

(2500,)

In [36]:
summed_spam_tokens.tail()

2495    104
2496    114
2497     88
2498     85
2499    167
dtype: int64

# Summing the Tokens occurring in Ham

In [37]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [38]:
summed_ham_tokens.shape

(2500,)

In [39]:
summed_ham_tokens.tail()

2495    79
2496    75
2497    98
2498    78
2499    12
dtype: int64

In [40]:
train_ham_tokens[2499].sum() + 1

np.int64(12)

# P(Token | Spam) - Probability that a Token occurs given that the Email is Spam|

In [41]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.000004
1    0.000240
2    0.009611
3    0.007783
4    0.005219
dtype: float64

In [42]:
prob_tokens_spam.sum()

np.float64(1.0)

# P(Token | Ham) - Probability that a Token occurs given that the Email is Non-Spam

In [43]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

np.float64(1.0)

# P(Token) - Probability that Token occurs

In [44]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [45]:
prob_tokens_all.sum()

np.float64(1.0)

# Save the Trained Model

In [46]:
TOKEN_SPAM_PROB_FILE = "token_spam_prob.txt"
TOKEN_HAM_PROB_FILE = "token_ham_prob.txt"
TOKEN_ALL_PROB_FILE = "token_all_prob.txt"

np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [47]:
sparse_test_data.shape

(589487, 4)

In [48]:
%%time

full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

C:\Users\rachi\AppData\Local\Temp\ipykernel_15048\630567689.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace = True)


CPU times: total: 25.9 s
Wall time: 34.2 s


In [49]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [50]:
TEST_FEATURE_MATRIX = 'test-features.txt'
TEST_TARGET_FILE = 'test-target.txt'

np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)